In [21]:
FILENAME = 'W2901173781.json'
EXPERIMENT_NAME = 'gemini-1.5-pro'
MANIFEST_CSV = 'records-48.csv'

import json
import Levenshtein
import csv
import math

POINTS_PER_AUTHOR_FIELD = 10
POINTS_PER_KEYWORD_FIELD = 10

def evaluate_authors(control_authors, experiment_authors):
    length = len(control_authors)
    
    possible_points = length * 2 * POINTS_PER_AUTHOR_FIELD # Possible points is 10 points per each name part (first/last). 
    
    total = (len(experiment_authors) - length) * 2 * POINTS_PER_AUTHOR_FIELD # account for extra authors found.
    
    # Question: Is the order of authors meaningful? Currently we are ensuring it is.
    for i in range(length):
        if len(experiment_authors) <= i:
            print(f"AUTHOR WAS NOT FOUND IN EXPERIMENT {i}")
        else:
            if experiment_authors[i]['first'] is not None:
                total += POINTS_PER_AUTHOR_FIELD - Levenshtein.distance(control_authors[i]['first'], experiment_authors[i]['first'])
            else:
                total -= POINTS_PER_AUTHOR_FIELD
            if experiment_authors[i].get('last') is not None:
                total += POINTS_PER_AUTHOR_FIELD - Levenshtein.distance(control_authors[i]['last'], experiment_authors[i]['last'])
            else:
                total -= POINTS_PER_AUTHOR_FIELD
    return total / possible_points

def evaluate_string(control_string, experiment_string):
    if (control_string is None and experiment_string) or (control_string and experiment_string is None):
        return 0;
    if (control_string is None and len(experiment_string) == 0):
        return 1;

    possible_points = len(control_string)
    return (possible_points - Levenshtein.distance(control_string, experiment_string)) / possible_points
    
def evaluate_keywords(control_keywords, experiment_keywords):
    length = len(control_keywords)

    if length == 0:
      if len(experiment_keywords) == 0:
        return 1.0
      else:
        return 0
    
    possible_points = length * POINTS_PER_KEYWORD_FIELD # Possible points is 10 points per keyword. 
    
    total = (len(experiment_keywords) - length) * POINTS_PER_KEYWORD_FIELD # account for extra keywords found.
    
    # Question: Is the order of keywords meaningful? Currently we are ensuring it is.
    for i in range(length):
        if len(experiment_keywords) <= i:
            print(f"KEYWORD WAS NOT FOUND IN EXPERIMENT {i}")
        else:
            if experiment_keywords[i] is not None:
                total += POINTS_PER_KEYWORD_FIELD - Levenshtein.distance(control_keywords[i], experiment_keywords[i])
            else:
                total -= POINTS_PER_KEYWORD_FIELD

    return total / possible_points

def evaluate_result(filename, debug=False):
    CONTROL = f'control/{filename}'
    EXPERIMENT = f'{EXPERIMENT_NAME}/results/{filename}'

    with open(CONTROL, 'r') as file:
      control = json.load(file)

    with open(EXPERIMENT, 'r') as file:
      experiment = json.load(file)

    author_score = evaluate_authors(control['authors'], experiment.get('authors', []))
    title_score = evaluate_string(control['title'], experiment['title'])
    abstract_score = evaluate_string(control['abstract'], experiment.get('abstract'))
    keyword_score = evaluate_keywords(control['keywords'], experiment.get('keywords', []))

    if debug:
        print('Author Score:', author_score)
        if author_score != 1.0:
          print("Expected Authors:", control['authors'])
          print("Model Authors:", experiment['authors'])
        
        print('Title Score:', title_score)
        if title_score != 1.0:
          print("Expected Title:", control['title'])
          print("Model Title:", experiment['title'])
        
        print('Abstract Score:', abstract_score)
        if abstract_score != 1.0:
          print("Expected Abstract:", control['abstract'])
          print("Model Abstract:", experiment['abstract'])
        
        print('Keyword Score:', keyword_score)
        if keyword_score != 1.0:
          print("Expected Keywords:", control['keywords'])
          print("Model Keywords:", experiment.get('keywords', []))

    return (author_score + title_score + abstract_score + keyword_score) /4


histogram = { '100': 0, '90':0, '80': 0, '70': 0, '60':0, '50':0, '40': 0, '20':0, '10':0 }
count = 0
with open(MANIFEST_CSV, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        id = row['OpenAlex ID'].removeprefix('https://openalex.org/')
        print(id)
        score = evaluate_result(f'{id}.json', debug=False)
        print('Total Score:', score)

        histogram[str(math.floor(score * 10)) + '0'] += 1
        count += 1
        print("\n")

print("Total:", count)
for key, value in histogram.items():
    print(key, value, f"({round(value/count, 2)*100}%)")



W2988715931
Total Score: 0.9940622440622441


W3202546816
Total Score: 1.0


W4212832069
Total Score: 1.0


W2901173781
Total Score: 0.8246191507643836


W3013783484
Total Score: 0.90625


W4226140866
Total Score: 0.8026014555619818


W4399283731
Total Score: 0.9987733071638861


W3126527592
Total Score: 1.0


W3093048371
Total Score: 0.5


W4205859241
Total Score: 0.9995364647713226


W4322154575
Total Score: 0.9847501995087053


W4297341531
Total Score: 0.9916431095406361


W4393156923
Total Score: 0.9989925158318941


W3000588783
Total Score: 0.9997881355932203


W3091235616
Total Score: 0.7296875


W4296103990
Total Score: 0.9969793887704336


W3170856023
Total Score: 1.0


W3164284701
Total Score: 1.0


W3183339884
Total Score: 0.9989440337909187


W4393028888
Total Score: 0.9921875


W4394813096
Total Score: 0.999766355140187


W4304614191
Total Score: 1.0


W4386128506
Total Score: 0.9997607655502392


W4376255784
Total Score: 1.0


W3177095392
Total Score: 0.9982197355035605


